# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of a *C++* library.

First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

* to detect the version of *Python* installed and to save it in the `PYTHON_VERSION` environment variable.

In [2]:
import os
import sys
os.environ['PYTHON_VERSION'] = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

* to import **AutoWIG**.

In [3]:
import autowig

Then, we need to install and compile the *C++* library.
To do so, we use available **Conda** recipes.

In [4]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\..\bin\conda\libbasic -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../../bin/conda/libbasic -c statiskit -c conda-forge
!conda install -y libbasic --use-local -c statiskit -c conda-forge --force

BUILD START: libbasic-1.0.0-0
    (actual version deferred until further download or env creation)

The following NEW packages will be INSTALLED:

    ca-certificates: 2017.7.27.1-0      conda-forge
    certifi:         2017.7.27.1-py27_0 conda-forge
    libtoolchain:    1.0.0-py27_0       local      
    ncurses:         5.9-10             conda-forge
    openssl:         1.0.2l-0           conda-forge
    path.py:         10.3.1-py27_0      conda-forge
    pip:             9.0.1-py27_0       conda-forge
    python:          2.7.14-0           conda-forge
    python-scons:    3.0.0-py27_0       local      
    pyyaml:          3.12-py27_1        conda-forge
    readline:        6.2-0              conda-forge
    scons-tools:     1.0.0-py27_0       local      
    setuptools:      36.3.0-py27_0      conda-forge
    six:             1.11.0-py27_1      conda-forge
    sqlite:          3.13.0-1           conda-forge
    tk:              8.5.19-2           conda-forge
    wheel:           

Fetching package metadata .................
Solving package specifications: .

Package plan for installation in environment /Users/pfernique/.miniconda/envs/FP17:

The following NEW packages will be INSTALLED:

    libbasic: 1.0.0-0 local



Once these preliminaries are done, we can proceed to the actual generation of wrappers for this *C++* library.
For this, we create an empty Abstract Semantic Graph (ASG).

In [5]:
asg = autowig.AbstractSemanticGraph()

We then parse the headers of this *C++* library with relevant compilation flags.

In [6]:
%%time

try:
    from path import path as Path
except:
    from path import Path
prefix = Path(sys.prefix).abspath()
if is_windows:
    headers = [prefix/'Library'/'include'/'basic'/'overload.h',
               prefix/'Library'/'include'/'basic'/'binomial.h']
else:
    headers = [prefix/'include'/'basic'/'overload.h',
               prefix/'include'/'basic'/'binomial.h']
    
flags = ['-x', 'c++', '-std=c++11']
if is_windows:
    flags.append('-I' + str((prefix/'Library'/'include').abspath()))
else:
    flags.append('-I' + str((prefix/'include').abspath()))
    
asg = autowig.parser(asg, headers,
                          flags,
                          silent = True)

CPU times: user 864 ms, sys: 118 ms, total: 982 ms
Wall time: 1.31 s


Since most of **AutoWIG** guidelines are respected in this *C++* library, the `default` `controller` implementation is suitable.

In [7]:
%%time

asg = autowig.controller(asg)

CPU times: user 361 ms, sys: 9.52 ms, total: 370 ms
Wall time: 364 ms


In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [8]:
%%time

autowig.generator.plugin = 'boost_python_internal'  
wrappers = autowig.generator(asg,
                             module = Path('.')/'..'/'..'/'src'/'py'/'_basic.cpp',
                             decorator = Path('.')/'..'/'..'/'src'/'py'/'basic'/'_basic.py',
                             prefix = 'wrapper_')

CPU times: user 66.9 ms, sys: 1.82 ms, total: 68.8 ms
Wall time: 67.4 ms


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [9]:
%%time

wrappers.write()

CPU times: user 558 ms, sys: 605 ms, total: 1.16 s
Wall time: 10.5 s


Here is the list of the generated wrappers (untracked files).

In [10]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   index.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/
	../../src/py/_basic.cpp
	../../src/py/_basic.h
	../../src/py/basic/_basic.py
	../../src/py/wrapper_0f744e8d056f5d469a887c7c78eaf8fe.cpp
	../../src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp
	../../src/py/wrapper_92d464d1243554009adc48a065a5be3e.cpp
	../../src/py/wrapper_f926cb231a7f5da09f313cd361ff94c7.cpp

no changes added to commit (use "git add" and/or "git commit -a")


And here, we present the wrappers generated for the `BinomialDistribution` class.

In [11]:
if is_windows:
    !pygmentize ..\..\src\py\wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp
else:
    !pygmentize ../../src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp

#include "_basic.h"



namespace autowig
{

}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::BinomialDistribution const volatile * get_pointer<class ::BinomialDistribution const volatile >(class ::BinomialDistribution const volatile *c) { return c; }
}
    #endif
#endif



void wrapper_4046a8421fe9587c9dfbc97778162c7d()
{

    double  (::BinomialDistribution::*method_pointer_3a3ff64f25e358a6a10b1cd3b3425b82)(unsigned int const) const = &::BinomialDistribution::pmf;
    double  (::BinomialDistribution::*method_pointer_3c97a500c9575c259d5cbdd76120ff4f)() const = &::BinomialDistribution::get_pi;
    void  (::BinomialDistribution::*method_pointer_d364a0529e33516f8ecbb7dcedd60aa0)(double const) = &::BinomialDistribution::set_pi;
    boost::python::class_< class ::BinomialDistribution, autowig::Held< class ::BinomialDistribution >::Type > class_4046a8421fe9587c9dfbc97778162c7d("BinomialDistribution", "", boost::python::no_init);
    class_4046a842

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [12]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\..\bin\conda\python-basic -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../../bin/conda/python-basic -c statiskit -c conda-forge
!conda install -y python-basic --use-local -c statiskit --force

BUILD START: python-basic-1.0.0-py27_0
    (actual version deferred until further download or env creation)

The following NEW packages will be INSTALLED:

    alabaster:                0.7.10-py27_1      conda-forge
    asn1crypto:               0.22.0-py27_0      conda-forge
    babel:                    2.4.0-py27_0       conda-forge
    ca-certificates:          2017.7.27.1-0      conda-forge
    certifi:                  2017.7.27.1-py27_0 conda-forge
    cffi:                     1.10.0-py27_0      conda-forge
    chardet:                  3.0.4-py27_0       conda-forge
    coverage:                 4.4.1-py27_0       conda-forge
    cryptography:             2.0.3-py27_0       conda-forge
    docutils:                 0.14-py27_0        conda-forge
    enum34:                   1.1.6-py27_1       conda-forge
    freetype:                 2.7-1              conda-forge
    icu:                      58.1-1             conda-forge
    idna:                     2.6-py27_1         co

g++ -o /Users/pfernique/.miniconda/conda-bld/python-basic_1507558796846/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/lib/libbasic.dylib -dynamiclib build/cpp/binomial.os build/cpp/overload.os -L/Users/pfernique/.miniconda/conda-bld/python-basic_1507558796846/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/lib -flat_namespace -undefined suppress
Install file: "src/py/basic/__init__.py" as "/Users/pfernique/.miniconda/conda-bld/python-basic_1507558796846/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/lib/python2.7/site-packages/basic/__init__.py"
Install file: "s


install_name_tool -add_rpath @loader_path/../../../ /Users/pfernique/.miniconda/conda-bld/python-basic_1507558796846/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/lib/python2.7/site-packages/basic/__basic.so

Fixing permissions
/Users/pfernique/.miniconda/conda-bld/osx-64/python-basic-1.0.0-py27_0.tar.bz2
updating index in: /Users/pfernique/.miniconda/conda-bld
updating index in: /Users/pfernique/.miniconda/conda-bld/noarch
updating index in: /Users/pfernique/.miniconda/conda-bld/osx-64
updating: python-basic-1.0.0-py27_0.tar.bz2
updating index in: /Users/pfernique/.miniconda/conda-bld
TEST START: /Users/pfernique/.miniconda/conda-bld/osx-64/python-basic-1.0.0-py27_0.tar.bz2
Deleting work directory, /Users/pfernique/.miniconda/conda-bld/python-basic_1507558796846/work

The following NEW packages will be INSTALLED:

    ca-certificates: 2017.7

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [13]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [14]:
binomial.pmf(0)

0.5

In [15]:
binomial.pmf(1)

0.5

In [16]:
binomial.n = 0
binomial

In [17]:
binomial.pmf(0)

1.0

In [18]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print(error)
else:
    raise Exception('A `basic.ProbabilityError` should have been raised')

a probability must be in the interval [0,1]


Here is a report concerning objects wrapped using this notebook.

In [19]:
import fp17
fp17.report(asg)

Headers: 3 (374 SLOC)
Variables: 1 (0.0%)
Functions: 15 (0.0%)
Classes: 4 (100.0%)
